In [1]:
import pandas as pd
import numpy as np

In [2]:
google=pd.read_csv('/Users/apple/Desktop/Python-Finance-QuantConnect/DATA/GOOG.csv', index_col='Date', parse_dates=True)
google.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4460 entries, 2004-08-20 to 2022-05-06
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       4460 non-null   float64
 1   High       4460 non-null   float64
 2   Low        4460 non-null   float64
 3   Close      4460 non-null   float64
 4   Adj Close  4460 non-null   float64
 5   Volume     4460 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 243.9 KB


In [12]:
google['DailyRet']=google['Adj Close'].pct_change(1)
google=google.dropna()
google

,Open,High,Low,Close,Adj Close,Volume,Daily Ret,DailyRet
Date,,,,,,,,
2004-08-24,55.412300,55.591629,51.591621,52.239197,52.239197,15319808,-0.041408,-0.041408
2004-08-25,52.284027,53.798351,51.746044,52.802086,52.802086,9232276,0.010775,0.010775
2004-08-26,52.279045,53.773445,52.134586,53.753517,53.753517,7128620,0.018019,0.018019
2004-08-27,53.848164,54.107193,52.647663,52.876804,52.876804,6241307,-0.016310,-0.016310
2004-08-30,52.443428,52.548038,50.814533,50.814533,50.814533,5221498,-0.039001,-0.039001
...,...,...,...,...,...,...,...,...
2022-05-02,2278.129883,2346.790039,2267.989990,2343.139893,2343.139893,1514000,0.019053,0.019053
2022-05-03,2335.300049,2386.000000,2332.540039,2362.590088,2362.590088,1060800,0.008301,0.008301
2022-05-04,2360.070068,2462.860107,2314.770020,2451.500000,2451.500000,1661600,0.037632,0.037632


In [13]:
google['DailyRet'].std()

0.019100573394156377

In [14]:
def compSR(df, rf=0):
    meanRet=df['DailyRet'].mean()
    stdDev=df['DailyRet'].std()
    SR=((meanRet-rf)/stdDev)
    return SR

In [16]:
GoogleSR=compSR(google)

In [18]:
GoogleSR

0.05350727325319511

Google Sharpe Ratio

In [19]:
def Sortino(df, threshold=0,rf=0.0):
    meanRet=df['DailyRet'].mean()
    downside_risk=df[df['DailyRet']<threshold]['DailyRet']
    stdDev=downside_risk.std()
    SortinoRatio=(meanRet-rf)/stdDev
    return SortinoRatio* (252**0.5)

In [20]:
GoogleSortino=Sortino(google)
GoogleSortino

1.212829962322287

Google Sortino Ratio

In [21]:
import scipy.stats

In [22]:
def PSR(df, benchmark=0):
    sr=compSR(df)
    skew=scipy.stats.skew(df['DailyRet'])
    kurtosis=scipy.stats.kurtosis(df['DailyRet'], fisher=True)
    n=len(df)
    sigmaSR=( (1/(n-1)) * ( 1+ 0.5 *sr**2  +skew*sr+ (kurtosis/4) *sr**2)  )**0.5
    finalr= (sr-benchmark)/sigmaSR
    psr=scipy.stats.norm.cdf(finalr)
    return psr

In [23]:
GooglePSR=PSR(google)
GooglePSR* (252**.5)

15.870657086837394

Google Probabilistic Sharpe Ratio